<a href="https://colab.research.google.com/github/pragmatic-ninja/human_pose/blob/master/Human_Pose_Module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVHN Dataset

In [1]:
!ls

sample_data
